In [ ]:
import os
import requests
import ollama
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

In [2]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [4]:
cnn_news = Website("https://cnn.com")
print(cnn_news.title)
print(cnn_news.text)

Breaking News, Latest News and Videos | CNN
CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
More
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
Watch
Listen
Live TV
Subscribe
Sign in
My Account
Settings
Newsletters
Topics yo

In [5]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [6]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [7]:
print(user_prompt_for(cnn_news))

You are looking at a website titled Breaking News, Latest News and Videos | CNN
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

CNN values your feedback
1. How relevant is this ad to you?
2. Did you encounter any technical issues?
Video player was slow to load content
Video content never loaded
Ad froze or did not finish loading
Video content did not start after ad
Audio on ad was too loud
Other issues
Ad never loaded
Ad prevented/slowed the page from loading
Content moved around while ad loaded
Ad was repetitive to ads I've seen previously
Other issues
Cancel
Submit
Thank You!
Your effort and contribution in providing this feedback is much
                                        appreciated.
Close
Ad Feedback
Close icon
US
World
Politics
Business
Health
Entertainment
Style
Travel
Sports
Science
Climate
Weather
Ukraine-Russia War
Israel-Hamas War
Games
More
US
World


In [8]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [9]:
messages_for(cnn_news)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': "You are looking at a website titled Breaking News, Latest News and Videos | CNN\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nCNN values your feedback\n1. How relevant is this ad to you?\n2. Did you encounter any technical issues?\nVideo player was slow to load content\nVideo content never loaded\nAd froze or did not finish loading\nVideo content did not start after ad\nAudio on ad was too loud\nOther issues\nAd never loaded\nAd prevented/slowed the page from loading\nContent moved around while ad loaded\nAd was repetitive to ads I've seen previously\nOther issues\nCancel\nSubmit\nThank You!\nYour effort and contribution in providing thi

In [ ]:
def summarize(url):
    website = Website(url)
    # response = ollama.chat(model="llama3.2", messages=messages_for(website))
    # return response['message']['content']
    response = openai.chat.completions.create(
        model = "llama3.2",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [11]:
summarize("https://cnn.com")

'**Summary**\n================================\n\nThe Breaking News, Latest News and Videos | CNN website provides comprehensive coverage of international news, business updates, entertainment information, sports highlights, and in-depth features. The site offers various sections such as US Politics, Business, World, Science, Climate and more.\n\n### Featured Stories\n\n- **Russia attacks Ukraine with one of the largest aerial assaults of the war**, killing four and injuring dozens.\n- **Trump\'s war on the arts**, many museums remain silent despite going after the arts; Trump’s behavior towards art critics is getting more aggressive.\n- **Prince William says 2024 was the hardest year of his life** for his mental health, stating that it has given him a new perspective on life.\n\n### Top Stories\n\n1. **Selena Gomez weds Benny Blanco**, after years of engagement they have finally gotten married in an intimate ceremony.\n2. **Nathan Howard/Pool/Reuters - Israel is becoming a pariah on t

In [12]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [13]:
display_summary("https://cnn.com")

**Summary of CNN Website**
=====================================

The Breaking News, Latest News and Videos | CNN website provides up-to-date news coverage on various topics including politics, business, health, entertainment, and more.

**Recent Headlines**
--------------------

* Russia attacks Ukraine with one of the largest aerial assaults of the war, killing four and injuring dozens
* Moldovans cast ballots in a tense election plagued by Russian interference claims
* Israeli strikes in Syria leave the country "stunned" and make normalization "difficult," Syrian foreign minister says
* Prince William says 2024 was the hardest year of his life
* Nearly undetectable threat of snipers has political security experts sounding alarms

**Featured Sections**
--------------------

* **USA Today**: Latest news on U.S. politics, including Trump's war on the arts and the government shutdown looms
* **Business**: China's electric car makers face a boom that has become a bloodbath
* **Health**: Acetaminophen use during pregnancy not linked to autism in study of 2.5 million children
* **Entertainment**: Selena Gomez gets married, and Meryl Streep crashes Milan fashion show for "Devil Wears Prada 2" shoot

**Popular Videos**
------------------

* CNN video shows Ukrainian air defenses intercepting Russian attack over Kyiv on Sunday morning
* Woman describesKyiv attack that destroyed her home
* Ryder Cup: American fans crank up heckling of Team Europe at Bethpage Black
* Prince William says 2024 was the hardest year of his life

In [14]:
display_summary("https://anthropic.com")

# Anthropic Website Summary
## Overview

Anthropic is a public benefit corporation dedicated to securing the benefits and mitigating the risks of AI. The website provides information on their research, economic futures, commitments, initiatives, transparency, and responsible scaling policy.

### Key Areas of Focus

*   **AI Safety**: Anthropic aims to design powerful technologies with human benefit at their foundation. They focus on building tools like Claude to show what responsible AI development looks like in practice.
*   **Anthropic Economic Index**: The index provides a clearer picture of how AI is being incorporated into real-world tasks across the modern economy.
*   **Claude**: A large language model that Anthropic builds to serve humanity's long-term well-being.

### News and Announcements

Anthropic has released several updates and announcements:

*   **Project Vend** (Aug 05, 2025): A new project aimed at exploring the societal impacts of AI.
*   **Claude Sonnet 4 with 1M context** (Sep 15, 2025): An update on Claude's capabilities.
*   **Clode Opus 4.1** (Aug 12, 2025): A new version of the Opus model.
*   **Introduction to Claude 4** (May 22, 2025): An announcement about a new update to Claude.

### Products and Pricing

Anthropic offers several products, including:

*   **Claude**: A large language model with various pricing plans
*   **Claude Code**: A developer platform for building with Claude
*   **Max Plan**, **Team Plan**, and **Enterprise Plan** (Pricing)

### Education and Resources

Anthropic provides guidance on:

*   **Responsible Scaling Policy**
*   **Anthropic Academy**: Learn to build with Claude
*   **Developer docs**: Find resources for developers working with their products